# 3.0 Cluster Analysis POI Df

In [1]:
# Standard libraries - run pip install if necessary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.ticker as ticker
from sklearn.decomposition import PCA

from datetime import datetime

# Geospatial libraries
from h3 import h3 
import geopandas as gp
import folium
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon
## Color for map 
import branca
import branca.colormap as cm

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import MiniBatchKMeans

# progress bar
from tqdm import tqdm
tqdm.pandas()

In [4]:
# Hexagon resolution to work with for the rest of the notebook
RES = 8

In [2]:
poi_df = pd.read_csv("data/prepped/poi_df.csv")
poi_df.head()

,osmid,amenity,name,geometry,public_transport,latitude,longitude,count,h3_res7,h3_res8,poly_res7,poly_res8,poi_density_res7,poi_density_res8,category
0,20217109,ferry_terminal,Shoreline Sightseeing,POINT (-87.6225172 41.8891445),station,41.889145,-87.622517,45,872664c1effffff,882664c1e3fffff,POLYGON ((-87.63048927308355 41.90755371098675...,POLYGON ((-87.62100146715468 41.89255291202126...,1621,328,Transportation
1,20217442,ferry_terminal,Union Station/Willis Tower - Shoreline Water T...,POINT (-87.6377402 41.8790618),station,41.879062,-87.637740,45,872664c1affffff,882664c1adfffff,POLYGON ((-87.63912440648137 41.88713767856642...,POLYGON ((-87.63912440648137 41.88713767856642...,1822,417,Transportation
2,269449042,parking_entrance,NaN,POINT (-87.6150579 41.8586894),NaN,41.858689,-87.615058,10,872664c1bffffff,882664c1b1fffff,"POLYGON ((-87.61820944356228 41.8710984903598,...",POLYGON ((-87.61347038938916 41.86360034164272...,349,36,Transportation
3,269450074,parking_entrance,NaN,POINT (-87.5841968 41.7917424),NaN,41.791742,-87.584197,10,872664cc5ffffff,882664cc59fffff,POLYGON ((-87.5937067068291 41.798228672497444...,POLYGON ((-87.58229602405846 41.79718226760902...,385,47,Transportation
4,269450344,parking_entrance,NaN,POINT (-87.6119452 41.8496767),NaN,41.849677,-87.611945,10,872664c1bffffff,882664c1b3fffff,"POLYGON ((-87.61820944356228 41.8710984903598,...",POLYGON ((-87.60873281770539 41.85610258554422...,349,26,Transportation


In [5]:
columns_to_keep = [f'h3_res{RES}', 'category']
poi_df_cluster = poi_df[columns_to_keep].copy()

# One-hot encode the different categories & then group by the hex res defined in beginning
poi_df_cluster = pd.get_dummies(poi_df_cluster, columns=['category'], prefix='', prefix_sep='')

poi_df_cluster = poi_df_cluster.groupby(f'h3_res{RES}').sum().reset_index()
poi_df_cluster.head()

,h3_res8,Accommodation,Animal Services,Education,Financial Services,Food and Drink,Healthcare,Miscellaneous,Personal Care,Public Services,Recreation and Entertainment,Religious and Community,Shopping and Retail,Sports and Fitness,Transportation,Utilities
0,8826641903fffff,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
1,8826641907fffff,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,8826641909fffff,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,882664190bfffff,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0
4,882664190dfffff,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
